In [1]:
!pip install discord.py
!pip install nest_asyncio

In [1]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
import torch
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision.utils import save_image
import random
import numpy as np

In [2]:
from google.colab import drive

drive.mount("/content/gdrive")
import os
sample_dir = 'Anime'
os.makedirs(sample_dir, exist_ok=True)


Mounted at /content/gdrive


In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    #np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [4]:
def convert_to_int(string):
  out = ''.join(str(ord(c)) for c in string)
  return int(out)


In [5]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [6]:
latent_size = 512
generator = nn.Sequential(
    # in: latent_size x 1 x 1
    nn.ConvTranspose2d(latent_size, 2048, kernel_size=4, stride=1, padding=0, bias=False),
    nn.BatchNorm2d(2048),
    nn.ReLU(True),

    nn.ConvTranspose2d(2048, 1024, kernel_size=4, stride=1, padding=0, bias=False),
    nn.BatchNorm2d(1024),
    nn.ReLU(True),
    # out: 512 x 4 x 4

    nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(512),
    nn.ReLU(True),
    # out: 256 x 8 x 8

    nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.ReLU(True),
    # out: 128 x 16 x 16

    nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.ReLU(True),
    # out: 64 x 32 x 32

    nn.ConvTranspose2d(128, 3, kernel_size=4, stride=2, padding=1, bias=False),
    nn.Tanh()
    # out: 3 x 64 x 64
)
generator.load_state_dict(torch.load('gdrive/MyDrive/Models/epoch_new-5.pt'))
generator.eval()

stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
def denorm(img_tensors):
    return img_tensors * stats[1][0] + stats[0][0]
def save_sample(file_name, latent_tensors):
    fake_images = generator(latent_tensors)[0]
    save_image(denorm(fake_images), os.path.join(sample_dir, file_name))

In [7]:
device = get_default_device()
generator = to_device(generator, device)
device

device(type='cuda')

In [ ]:
import discord
import asyncio
import nest_asyncio


nest_asyncio.apply()


async def main():
  TOKEN = 'INSERT YOUR TOKEN HERE'

  #client = discord.Client()

  from discord.ext import commands

  bot = commands.Bot(command_prefix='<', help_command=None)


  @bot.event
  async def on_ready():
     print('Ready')

  @bot.command(pass_context=True)
  @commands.cooldown(4, 1, commands.BucketType.user)
  async def help(ctx):

      embed = discord.Embed(
          colour=discord.Colour.from_rgb(153, 102, 51),
          title="Fuuka",
          description="Deep learning shit bot"
      )

      #embed.set_author(name="Author", icon_url="https://cdn.discordapp.com/attachments/755435067282555011/781253419486150666/fuuka.png")
      #embed.set_image(url="https://cdn.discordapp.com/attachments/755435067282555011/781254299908702208/puuka.png")
      embed.set_thumbnail(url="https://cdn.discordapp.com/attachments/755435067282555011/781254299908702208/puuka.png")
      embed.add_field(name="ping", value="This has the bot say pong", inline=False)
      embed.add_field(name="cum", value="Allows to cum on someone.\n Usage: cum <USER NAME>", inline=False)
      embed.add_field(name="help", value="Calls help", inline=False)
      embed.add_field(name="anime", value="Generates anime picture\n Usage: anime <optional_seed>", inline=False)
      embed.set_footer(text='Fuuka is a "Yotsubato!" manga character')

      await ctx.send(embed=embed)


  @bot.command(name="ping", description="pong")
  @commands.cooldown(4, 1, commands.BucketType.user)
  async def ping(ctx):
    await ctx.send('Pong! {0}'.format(round(bot.latency, 6)))

  @bot.command(name="anime", description="Post generated anime picture")
  @commands.cooldown(4, 1, commands.BucketType.user)
  async def anime(ctx, seed = None):
    seed_str = str(seed)
    if seed != None:
      seed_everything(convert_to_int(seed_str))

    rand = torch.randn(1, latent_size, 2, 2, device=device) # random latent tensors
    save_sample('example.png', rand)
    await ctx.send(file=discord.File('Anime/example.png'))
    

  @bot.command(name="cum", description="Allows to cum on someone. Usage: cum <USER NAME>")
  @commands.cooldown(4, 1, commands.BucketType.user)
  async def cum(ctx, username = None):
    guy = ctx.message.author.display_name
    if username == None:
        await ctx.send('{} cummed inside yourself'.format(guy))
        return
    slut = username
    await ctx.send('{} cummed on {}'.format( guy, slut))





  @ping.error
  @cum.error
  @anime.error
  async def error(ctx, error):
    if isinstance(error, commands.CommandOnCooldown):
        msg = 'Coldown'.format(error.retry_after)
        await ctx.send(msg, delete_after=1) 
    else:
        raise error
    


    
  bot.run(TOKEN)


asyncio.run(main())


